# Q2: What is the variability of albedo within polygonal tundra? 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

# Variability of Observed (Tram) Albedo

In [2]:
tram_data = pd.read_csv("../data/tram_complete.csv")

In [3]:
tram_data['Stop_number'].max()

137

In [4]:
tram_data.columns

Index(['Unnamed: 0', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second',
       'DD_AKST', 'DD_Expected_AKST', 'Offset_time', 'DD_UTC', 'Run_number',
       'Stop_number', 'Height', 'ShortW_inc', 'ShortW_out', 'Albedo',
       'LongW_inc', 'LongW_out', 'Net_shortW', 'Net_longW', 'Net_Rad',
       'Sol_Irr_inc', 'Sol_Irr_out', 'PAR_inc', 'PAR_out', 'fPAR', 'Red_i',
       'NIR_i', 'Red_r', 'NIR_r', 'NDVI', 'T_air_C', 'T_surf_C', 'date',
       'Polygon_type', 'Topo_feature'],
      dtype='object')

In [38]:
print(len(tram_data[(tram_data['Polygon_type'] == 'LCP') & (tram_data['Topo_feature'] == 'center')]['Stop_number'].unique()))
print(len(tram_data[(tram_data['Polygon_type'] == 'LCP') & (tram_data['Topo_feature'] == 'trough')]['Stop_number'].unique()))
print(len(tram_data[(tram_data['Polygon_type'] == 'LCP') & (tram_data['Topo_feature'] == 'rim')]['Stop_number'].unique()))


print(len(tram_data[(tram_data['Polygon_type'] == 'FCP') & (tram_data['Topo_feature'] == 'center')]['Stop_number'].unique()))
print(len(tram_data[(tram_data['Polygon_type'] == 'FCP') & (tram_data['Topo_feature'] == 'trough')]['Stop_number'].unique()))
print(len(tram_data[(tram_data['Polygon_type'] == 'FCP') & (tram_data['Topo_feature'] == 'rim')]['Stop_number'].unique()))


30
3
9
14
41
40


In [ ]:
print("% of Tram points = LCP: ", (len(tram_data[tram_data['Polygon_type'] == 'LCP'])/len(tram_data))*100)
print("% of Tram points = FCP: ", (len(tram_data[tram_data['Polygon_type'] == 'FCP'])/len(tram_data))*100)
print("% of Tram points = HCP: ", (len(tram_data[tram_data['Polygon_type'] == 'HCP'])/len(tram_data))*100)

% of Tram points = LCP:  30.35103272935091
% of Tram points = FCP:  69.64896727064908
% of Tram points = HCP:  0.0


In [6]:

print("% of Tram points = center: ", (len(tram_data[tram_data['Topo_feature'] == 'center'])/len(tram_data))*100)
print("% of Tram points = trough: ", (len(tram_data[tram_data['Topo_feature'] == 'trough'])/len(tram_data))*100)
print("% of Tram points = rim: ", (len(tram_data[tram_data['Topo_feature'] == 'rim'])/len(tram_data))*100)

% of Tram points = center:  31.412703413611954
% of Tram points = trough:  32.58501058739917
% of Tram points = rim:  36.00228599898889


In [7]:

print("% of Tram points = LCP center: ", (len(tram_data[(tram_data['Topo_feature'] == 'center') & (tram_data['Polygon_type'] == 'LCP')])/len(tram_data))*100)
print("% of Tram points = LCP trough: ", (len(tram_data[(tram_data['Topo_feature'] == 'trough') & (tram_data['Polygon_type'] == 'LCP')])/len(tram_data))*100)
print("% of Tram points = LCP rim: ", (len(tram_data[(tram_data['Topo_feature'] == 'rim') & (tram_data['Polygon_type'] == 'LCP')])/len(tram_data))*100)


print("% of Tram points = FCP center: ", (len(tram_data[(tram_data['Topo_feature'] == 'center') & (tram_data['Polygon_type'] == 'FCP')])/len(tram_data))*100)
print("% of Tram points = FCP trough: ", (len(tram_data[(tram_data['Topo_feature'] == 'trough') & (tram_data['Polygon_type'] == 'FCP')])/len(tram_data))*100)
print("% of Tram points = FCP rim: ", (len(tram_data[(tram_data['Topo_feature'] == 'rim') & (tram_data['Polygon_type'] == 'FCP')])/len(tram_data))*100)


% of Tram points = LCP center:  21.619542360587033
% of Tram points = LCP trough:  2.1826894191950648
% of Tram points = LCP rim:  6.54880094956881
% of Tram points = FCP center:  9.79316105302492
% of Tram points = FCP trough:  30.4023211682041
% of Tram points = FCP rim:  29.453485049420074


Spring snow defined on the tram as:
- 2014: May 27 - May 30
- 2016: April 21 - May 10, 
- 2017: April 25 - May 26 

Spring transition defined on the tram as:
- 2014: May 30 - June 25
- 2016: May 10 - June 3
- 2017: May 26 - June 20

Snow-free growing season defined on the tram as:
- 2014: June 25 - September 1 & September 8 - September 19
- 2015: June 8 - September 7
- 2016: June 10 - August 1
- 2017: June 20 - September 10

In [8]:
def calculate_albedo_stats(df):
    """
    Calculate mean and standard deviation of albedo, NDVI, and ShortW_inc for each polygon type and topographic feature.
    """
    albedo_means = df.groupby(['Polygon_type', 'Topo_feature'])['Albedo'].mean().unstack()
    albedo_means['Average'] = df.groupby('Polygon_type')['Albedo'].mean()
    albedo_means.loc['Average'] = df.groupby('Topo_feature')['Albedo'].mean()

    albedo_std = df.groupby(['Polygon_type', 'Topo_feature'])['Albedo'].std().unstack()
    albedo_std['Average'] = df.groupby('Polygon_type')['Albedo'].std()
    albedo_std.loc['Average'] = df.groupby('Topo_feature')['Albedo'].std()

    mean_albedo = df['Albedo'].mean()
    stdev_albedo = df['Albedo'].std()
    print(f"Albedo for this time is {mean_albedo:.3f} ± {stdev_albedo:.3f}")

    mean_ndvi = df['NDVI'].mean()
    stdev_ndvi = df['NDVI'].std()
    print(f"NDVI for this time is {mean_ndvi:.3f} ± {stdev_ndvi:.3f}")

    mean_t_air = df['T_air_C'].mean()
    stdev_t_air = df['T_air_C'].std()
    print(f"T_air_C for this time is {mean_t_air:.3f} ± {stdev_t_air:.3f}")


    return albedo_means, albedo_std


## Tram Spring Snow Variability

### All Years

In [9]:
# set date range for spring snow data
spring_snow_data = tram_data[((tram_data['date'] >= '2014-05-27') & (tram_data['date'] <= '2014-05-30')) | 
                                     ((tram_data['date'] >= '2016-04-21') & (tram_data['date'] <= '2016-05-10')) | 
                                     ((tram_data['date'] >= '2017-04-25') & (tram_data['date'] <= '2017-05-26'))]

In [10]:
albedo_spring_snow_means, albedo_spring_snow_std = calculate_albedo_stats(spring_snow_data)
print('Albedo means: ', albedo_spring_snow_means)
print('Albedo std:', albedo_spring_snow_std)

Albedo for this time is 0.830 ± 0.035
NDVI for this time is -0.112 ± 0.016
T_air_C for this time is -5.756 ± 4.482
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.832090  0.828572  0.828341  0.828962
LCP           0.832129  0.832026  0.834393  0.832268
Average       0.832117  0.829197  0.828744       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.032061  0.037567  0.033597  0.035152
LCP           0.033810  0.036018  0.029989  0.034038
Average       0.033278  0.037312  0.033400       NaN


### 2014

In [11]:
spring_snow_data_2014 = tram_data[((tram_data['date'] >= '2014-05-27') & (tram_data['date'] <= '2014-05-30'))]

albedo_spring_snow_means_2014, albedo_spring_snow_std_2014 = calculate_albedo_stats(spring_snow_data_2014)
print('Albedo means: ', albedo_spring_snow_means_2014)
print('Albedo std:', albedo_spring_snow_std_2014)

Albedo for this time is 0.791 ± 0.026
NDVI for this time is -0.117 ± 0.024
T_air_C for this time is -5.541 ± 0.801
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.806959  0.786208  0.790702  0.791038
LCP           0.790362  0.780607  0.809711  0.789654
Average       0.795380  0.785180  0.791998       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.022572  0.029173  0.024571  0.027205
LCP           0.020550  0.019427  0.019422  0.021329
Average       0.022497  0.027715  0.024712       NaN


### 2016

In [12]:
spring_snow_data_2016 = tram_data[((tram_data['date'] >= '2016-04-21') & (tram_data['date'] <= '2016-05-10'))]

albedo_spring_snow_means_2016, albedo_spring_snow_std_2016 = calculate_albedo_stats(spring_snow_data_2016)
print('Albedo means: ', albedo_spring_snow_means_2016)
print('Albedo std:', albedo_spring_snow_std_2016)

Albedo for this time is 0.812 ± 0.032
NDVI for this time is -0.110 ± 0.010
T_air_C for this time is -5.544 ± 2.910
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.816834  0.807229  0.810390  0.809973
LCP           0.817357  0.822093  0.824204  0.818861
Average       0.817182  0.809719  0.811225       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.029559  0.033325  0.031308  0.032075
LCP           0.028028  0.031607  0.031231  0.029125
Average       0.028532  0.033491  0.031458       NaN


### 2017

In [13]:
spring_snow_data_2017 = tram_data[((tram_data['date'] >= '2017-04-25') & (tram_data['date'] <= '2017-05-26'))]

albedo_spring_snow_means_2017, albedo_spring_snow_std_2017 = calculate_albedo_stats(spring_snow_data_2017)
print('Albedo means: ', albedo_spring_snow_means_2017)
print('Albedo std:', albedo_spring_snow_std_2017)

Albedo for this time is 0.841 ± 0.030
NDVI for this time is -0.112 ± 0.015
T_air_C for this time is -5.843 ± 5.113
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.840156  0.841090  0.839220  0.840143
LCP           0.842541  0.843204  0.840906  0.842566
Average       0.841818  0.841478  0.839334       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.030419  0.031635  0.028515  0.030145
LCP           0.029751  0.030195  0.028433  0.029752
Average       0.029971  0.031382  0.028509       NaN


## Tram Spring Transition Variability

### All years

In [14]:
spring_transition_data = tram_data[((tram_data['date'] > '2014-05-30') & (tram_data['date'] < '2014-06-25')) | 
                                           ((tram_data['date'] > '2016-05-10') & (tram_data['date'] < '2016-06-03'))| 
                                           ((tram_data['date'] > '2017-05-26') & (tram_data['date'] < '2017-06-20'))]

In [15]:
albedo_spring_transition_means, albedo_spring_transition_std = calculate_albedo_stats(spring_transition_data)
print('Albedo means: ',albedo_spring_transition_means)
print('Albedo std:', albedo_spring_transition_std)

Albedo for this time is 0.482 ± 0.233
NDVI for this time is 0.006 ± 0.208
T_air_C for this time is 0.035 ± 1.878
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.489227  0.431031  0.452636  0.448498
LCP           0.574746  0.519261  0.542757  0.560356
Average       0.548051  0.446888  0.458673       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.244312  0.230856  0.233328  0.234604
LCP           0.203459  0.217249  0.206939  0.208050
Average       0.220611  0.230957  0.232736       NaN


### 2014

In [16]:
spring_transition_data_2014 = tram_data[((tram_data['date'] > '2014-05-30') & (tram_data['date'] < '2014-06-25'))]
albedo_spring_transition_means_2014, albedo_spring_transition_std_2014 = calculate_albedo_stats(spring_transition_data_2014)
print('Albedo means: ',albedo_spring_transition_means_2014)
print('Albedo std:', albedo_spring_transition_std_2014)

Albedo for this time is 0.343 ± 0.187
NDVI for this time is 0.087 ± 0.228
T_air_C for this time is 0.766 ± 1.154
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.343487  0.283285  0.311151  0.303567
LCP           0.462978  0.333698  0.405609  0.431114
Average       0.427488  0.292558  0.317614       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.205586  0.145825  0.169634  0.166691
LCP           0.199207  0.166313  0.189647  0.199042
Average       0.208361  0.151039  0.172687       NaN


### 2016

In [17]:
spring_transition_data_2016 = tram_data[((tram_data['date'] > '2016-05-10') & (tram_data['date'] < '2016-06-03'))]
albedo_spring_transition_means_2016, albedo_spring_transition_std_2016 = calculate_albedo_stats(spring_transition_data_2016)
print('Albedo means: ', albedo_spring_transition_means_2016)
print('Albedo std:', albedo_spring_transition_std_2016)

Albedo for this time is 0.418 ± 0.219
NDVI for this time is 0.048 ± 0.212
T_air_C for this time is -0.310 ± 2.286
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.427510  0.359309  0.372623  0.374742
LCP           0.541478  0.495489  0.491774  0.527457
Average       0.503023  0.382752  0.380283       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.235693  0.193128  0.204997  0.205954
LCP           0.215352  0.207541  0.205732  0.213911
Average       0.228814  0.202293  0.207080       NaN


### 2017

In [18]:
spring_transition_data_2017 = tram_data[((tram_data['date'] > '2017-05-26') & (tram_data['date'] < '2017-06-20'))]
albedo_spring_transition_means_2017, albedo_spring_transition_std_2017 = calculate_albedo_stats(spring_transition_data_2017)
print('Albedo means: ', albedo_spring_transition_means_2017)
print('Albedo std:', albedo_spring_transition_std_2017)

Albedo for this time is 0.641 ± 0.175
NDVI for this time is -0.090 ± 0.142
T_air_C for this time is -0.215 ± 1.772
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.652217  0.604571  0.629654  0.622051
LCP           0.684812  0.678232  0.687610  0.683597
Average       0.674941  0.618061  0.633616       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.175210  0.200888  0.180916  0.189679
LCP           0.128299  0.116936  0.106843  0.124494
Average       0.144873  0.190463  0.177435       NaN


## Observed growing season variability

### All Years

In [19]:
growing_season_data = tram_data[((tram_data['date'] >= '2014-06-25') & (tram_data['date'] <= '2014-09-01')) |
                                 ((tram_data['date'] >= '2014-09-08') & (tram_data['date'] <= '2014-09-19')) |
                                 ((tram_data['date'] >= '2015-06-08') & (tram_data['date'] <= '2015-09-07')) |
                                 ((tram_data['date'] >= '2016-06-10') & (tram_data['date'] <= '2016-08-01')) |
                                 ((tram_data['date'] >= '2017-06-20') & (tram_data['date'] <= '2017-09-20'))]


In [20]:
albedo_growing_season_means, albedo_growing_season_std = calculate_albedo_stats(growing_season_data)
print('Albedo means: ',albedo_growing_season_means)
print('Albedo std:', albedo_growing_season_std)

Albedo for this time is 0.153 ± 0.039
NDVI for this time is 0.447 ± 0.117
T_air_C for this time is 5.400 ± 3.858
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.157191  0.169666  0.153683  0.160938
LCP           0.127244  0.151974  0.153874  0.134516
Average       0.136633  0.166442  0.153696       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.037669  0.035396  0.030910  0.034697
LCP           0.042386  0.031079  0.031127  0.041081
Average       0.043257  0.035316  0.030924       NaN


### 2014

In [21]:
growing_season_data_2014 = tram_data[((tram_data['date'] >= '2014-06-25') & (tram_data['date'] <= '2014-09-01')) |
                                 ((tram_data['date'] >= '2014-09-08') & (tram_data['date'] <= '2014-09-19'))]
albedo_growing_season_means_2014, albedo_growing_season_std_2014 = calculate_albedo_stats(growing_season_data_2014)
print('Albedo means: ', albedo_growing_season_means_2014)
print('Albedo std:', albedo_growing_season_std_2014)

Albedo for this time is 0.139 ± 0.040
NDVI for this time is 0.433 ± 0.097
T_air_C for this time is 4.141 ± 3.076
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.143149  0.157359  0.143357  0.149248
LCP           0.107145  0.135575  0.139333  0.115775
Average       0.118444  0.153322  0.143084       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.039490  0.035028  0.035002  0.036345
LCP           0.038731  0.030001  0.028949  0.038713
Average       0.042399  0.035184  0.034637       NaN


### 2015

In [22]:
growing_season_data_2015 = tram_data[(tram_data['date'] >= '2015-06-08') & (tram_data['date'] <= '2015-09-07')]
albedo_growing_season_means_2015, albedo_growing_season_std_2015 = calculate_albedo_stats(growing_season_data_2015)
print('Albedo means: ', albedo_growing_season_means_2015)
print('Albedo std:', albedo_growing_season_std_2015)

Albedo for this time is 0.163 ± 0.041
NDVI for this time is 0.498 ± 0.124
T_air_C for this time is 5.256 ± 3.501
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.164959  0.178838  0.161853  0.169389
LCP           0.139737  0.162354  0.162904  0.146284
Average       0.148079  0.175906  0.161922       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.038401  0.043888  0.030858  0.038728
LCP           0.043347  0.036228  0.038244  0.042809
Average       0.043427  0.043088  0.031390       NaN


### 2016

In [23]:
growing_season_data_2016 = tram_data[(tram_data['date'] >= '2016-06-10') & (tram_data['date'] <= '2016-08-01')]
albedo_growing_season_means_2016, albedo_growing_season_std_2016 = calculate_albedo_stats(growing_season_data_2016)
print('Albedo means: ', albedo_growing_season_means_2016)
print('Albedo std:', albedo_growing_season_std_2016)

Albedo for this time is 0.162 ± 0.030
NDVI for this time is 0.419 ± 0.112
T_air_C for this time is 5.120 ± 4.130
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.168161  0.177664  0.161152  0.169147
LCP           0.141078  0.162619  0.164221  0.147301
Average       0.149149  0.174895  0.161358       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.030592  0.021974  0.022668  0.024844
LCP           0.037406  0.018449  0.018876  0.034608
Average       0.037608  0.022149  0.022444       NaN


### 2017

In [24]:
growing_season_data_2017 = tram_data[(tram_data['date'] >= '2017-06-20') & (tram_data['date'] <= '2017-08-20')]
albedo_growing_season_means_2017, albedo_growing_season_std_2017 = calculate_albedo_stats(growing_season_data_2017)
print('Albedo means: ', albedo_growing_season_means_2017)
print('Albedo std:', albedo_growing_season_std_2017)

Albedo for this time is 0.150 ± 0.032
NDVI for this time is 0.414 ± 0.110
T_air_C for this time is 7.871 ± 4.085
Albedo means:  Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.155448  0.166414  0.149232  0.157470
LCP           0.124909  0.151426  0.152631  0.132588
Average       0.134104  0.163680  0.149466       NaN
Albedo std: Topo_feature    center       rim    trough   Average
Polygon_type                                        
FCP           0.032089  0.023443  0.024928  0.026660
LCP           0.038382  0.021493  0.021301  0.036467
Average       0.039188  0.023812  0.024709       NaN


# Variability of projected albedo

In [25]:
polygon_types = {
    1: 'Trough & LCP',
    2: 'Trough & FCP',
    3: 'Trough & HCP',
    4: 'Center & LCP',
    5: 'Center & FCP',
    7: 'High & LCP',
    8: 'High & FCP',
    9: 'High & HCP'
}


path_to_projections = "/Users/colettebrown/repos/tram/data/projections/"

group_order = ['_spring_snow', '_spring_transition', '_peak_NDVI']
variables = ['Albedo']

polygon_types = {
    1: 'Trough & LCP',
    2: 'Trough & FCP',
    3: 'Trough & HCP',
    4: 'Center & LCP',
    5: 'Center & FCP',
    7: 'High & LCP',
    8: 'High & FCP',
    9: 'High & HCP'
}

In [26]:
# Read in the projected data
all_data = pd.DataFrame()

for group in group_order:
    for variable in variables:
        file_pattern = os.path.join(path_to_projections, f"{variable}{group}.txt")
        for file in glob.glob(file_pattern):
            file_name = os.path.basename(file)
            
            # Extract the group and variable from the file name
            df = pd.read_csv(file, sep='\t')
            df['Group'] = group
            df['Variable'] = variable
            
            all_data = pd.concat([all_data, df], ignore_index=True)

# Clean up the data
# all_data.replace(polygon_types, inplace=True)

In [27]:
all_data

,e05,e06,e00,Var4,Group,Variable
0,585332.46,7909903.8,8.0,0.891,_spring_snow,Albedo
1,585332.46,7909904.3,8.0,0.826,_spring_snow,Albedo
2,585332.46,7909904.8,8.0,0.891,_spring_snow,Albedo
3,585332.46,7909905.3,8.0,0.893,_spring_snow,Albedo
4,585332.46,7909905.8,8.0,0.830,_spring_snow,Albedo
...,...,...,...,...,...,...
6151261,585992.96,7910676.8,NaN,NaN,_peak_NDVI,Albedo
6151262,585992.96,7910677.3,NaN,NaN,_peak_NDVI,Albedo
6151263,585992.96,7910677.8,NaN,NaN,_peak_NDVI,Albedo
6151264,585992.96,7910678.3,NaN,NaN,_peak_NDVI,Albedo


In [28]:
print("% of points = LCP: ", ((len(all_data[(all_data['e00'] == 1) | (all_data['e00'] == 4) | (all_data['e00'] == 7)]))/len(all_data))*100)
print("% of points = FCP: ",((len(all_data[(all_data['e00'] == 2) | (all_data['e00'] == 5) | (all_data['e00'] == 8)]))/len(all_data))*100)
print("% of points = HCP: ",((len(all_data[(all_data['e00'] == 3) | (all_data['e00'] == 9)]))/len(all_data))*100)

% of points = LCP:  23.745014440929722
% of points = FCP:  36.97385221188614
% of points = HCP:  6.895556134298209


In [29]:
print("% of points = LCP trough: ", ((len(all_data[all_data['e00'] == 1]))/len(all_data))*100)
print("% of points = FCP trough: ", ((len(all_data[all_data['e00'] == 2]))/len(all_data))*100)
print("% of points = HCP trough: ", ((len(all_data[all_data['e00'] == 3]))/len(all_data))*100)
print("% of points = LCP center: ", ((len(all_data[all_data['e00'] == 4]))/len(all_data))*100)
print("% of points = LCP center: ", ((len(all_data[all_data['e00'] == 5]))/len(all_data))*100)
print("% of points = LCP high: ", ((len(all_data[all_data['e00'] == 7]))/len(all_data))*100)
print("% of points = FCP high: ", ((len(all_data[all_data['e00'] == 8]))/len(all_data))*100)
print("% of points = HCP high: ", ((len(all_data[all_data['e00'] == 9]))/len(all_data))*100)


% of points = LCP trough:  3.4454370856340795
% of points = FCP trough:  10.852253828724038
% of points = HCP trough:  3.4545571594530298
% of points = LCP center:  8.431678942188487
% of points = LCP center:  7.645304234933102
% of points = LCP high:  11.867898413107156
% of points = FCP high:  18.476294148229
% of points = HCP high:  3.4409989748451784


In [30]:
def creating_albedo_table_all_data(df, day):
    '''
    Creates a table of the albedo on a specified day separated by polygon type and feature.

    Args: 
        df: a DataFrame of the projected albedo on a given day
        day: A string of one of the df['Day'].unique() options (spring_snow_, spring_transition_, spring_no_snow_, peak_NDVI_)

    Returns:
        pd.DataFrame: Df of a table containing columns: trough, center, high, overall and rows: LCP, FCP, trough, center, high
    '''

    print(f'Means ± Standard Deviations for {day}:')

    day_subset_data = df[df['Group'] == day]

    subset_data = day_subset_data[(day_subset_data['e00'] != 3) & (day_subset_data['e00'] != 9) & (day_subset_data['e00'].notnull())]

    means_stds = {
        "Trough": {
            "LCP": f"{subset_data.loc[subset_data['e00'] == 1, 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'] == 1, 'Var4'].std():.2f}",
            "FCP": f"{subset_data.loc[subset_data['e00'] == 2, 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'] == 2, 'Var4'].std():.2f}",
            "Trough": f"{subset_data.loc[subset_data['e00'].isin([1, 2]), 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'].isin([1, 2]), 'Var4'].std():.2f}"
        },
        "Center": {
            "LCP": f"{subset_data.loc[subset_data['e00'] == 4, 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'] == 4, 'Var4'].std():.2f}",
            "FCP": f"{subset_data.loc[subset_data['e00'] == 5, 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'] == 5, 'Var4'].std():.2f}",
            "Center": f"{subset_data.loc[subset_data['e00'].isin([4, 5]), 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'].isin([4, 5]), 'Var4'].std():.2f}"
        },
        "High": {
            "LCP": f"{subset_data.loc[subset_data['e00'] == 7, 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'] == 7, 'Var4'].std():.2f}",
            "FCP": f"{subset_data.loc[subset_data['e00'] == 8, 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'] == 8, 'Var4'].std():.2f}",
            "High": f"{subset_data.loc[subset_data['e00'].isin([7, 8]), 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'].isin([7, 8]), 'Var4'].std():.2f}"
        },
        "Overall": {
            "LCP": f"{subset_data.loc[subset_data['e00'].isin([1, 4, 7]), 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'].isin([1, 4, 7]), 'Var4'].std():.2f}",
            "FCP": f"{subset_data.loc[subset_data['e00'].isin([2, 5, 8]), 'Var4'].mean():.3f} ± {subset_data.loc[subset_data['e00'].isin([2, 5, 8]), 'Var4'].std():.2f}"
        }
    }

    df = pd.DataFrame(means_stds)
    print(df.to_string())
    print("\n")
    return(df)

In [31]:
# Create albedo tables for each day Group
days_for_analysis = all_data['Group'].unique()

for day in days_for_analysis:
    creating_albedo_table_all_data(all_data, day)

Means ± Standard Deviations for _spring_snow:
              Trough        Center          High       Overall
LCP     0.855 ± 0.03  0.859 ± 0.03  0.854 ± 0.03  0.856 ± 0.03
FCP     0.863 ± 0.03  0.862 ± 0.03  0.850 ± 0.03  0.856 ± 0.03
Trough  0.861 ± 0.03           NaN           NaN           NaN
Center           NaN  0.860 ± 0.03           NaN           NaN
High             NaN           NaN  0.852 ± 0.03           NaN


Means ± Standard Deviations for _spring_transition:
              Trough        Center          High       Overall
LCP     0.451 ± 0.14  0.523 ± 0.16  0.369 ± 0.19  0.436 ± 0.19
FCP     0.377 ± 0.20  0.373 ± 0.16  0.286 ± 0.14  0.331 ± 0.17
Trough  0.394 ± 0.19           NaN           NaN           NaN
Center           NaN  0.452 ± 0.18           NaN           NaN
High             NaN           NaN  0.318 ± 0.17           NaN


Means ± Standard Deviations for _peak_NDVI:
              Trough        Center          High       Overall
LCP     0.134 ± 0.01  0.112 ± 0.02 